이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Spark Session**을 하나 만든다

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark FS Partition Demo") \
        .master("local[3]") \
        .enableHiveSupport() \
        .getOrCreate()

In [3]:
!wget https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv

--2023-01-29 04:24:54--  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv
Resolving pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)... 52.218.228.225, 52.92.194.74, 52.218.220.193, ...
Connecting to pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)|52.218.228.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/csv]
Saving to: ‘appl_stock.csv.1’

appl_stock.csv.1    100%[===================>] 139.78K   765KB/s    in 0.2s    

2023-01-29 04:24:54 (765 KB/s) - ‘appl_stock.csv.1’ saved [143130/143130]



In [4]:
df = spark.read.csv("appl_stock.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [5]:
df.show(10)

+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|               Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:00|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|    

In [6]:
df = df.withColumn("year", year(df.Date)) \
    .withColumn("month", month(df.Date))

In [8]:
spark.sql("DROP TABLE IF EXISTS appl_stock")

DataFrame[]

In [9]:
df.write.partitionBy("year", "month").saveAsTable("appl_stock")

In [10]:
!ls -tl 

total 296
drwxr-xr-x 3 root root   4096 Jan 29 04:27 spark-warehouse
-rw-r--r-- 1 root root    670 Jan 29 04:26 derby.log
drwxr-xr-x 5 root root   4096 Jan 29 04:26 metastore_db
drwxr-xr-x 1 root root   4096 Jan 26 14:33 sample_data
-rw-r--r-- 1 root root 143130 Jul  6  2018 appl_stock.csv
-rw-r--r-- 1 root root 143130 Jul  6  2018 appl_stock.csv.1


In [11]:
!ls -tl spark-warehouse/

total 4
drwxr-xr-x 9 root root 4096 Jan 29 04:27 appl_stock


In [12]:
!ls -tl spark-warehouse/appl_stock/

total 28
-rw-r--r--  1 root root    0 Jan 29 04:27  _SUCCESS
drwxr-xr-x 14 root root 4096 Jan 29 04:27 'year=2016'
drwxr-xr-x 14 root root 4096 Jan 29 04:27 'year=2015'
drwxr-xr-x 14 root root 4096 Jan 29 04:27 'year=2014'
drwxr-xr-x 14 root root 4096 Jan 29 04:27 'year=2013'
drwxr-xr-x 14 root root 4096 Jan 29 04:27 'year=2012'
drwxr-xr-x 14 root root 4096 Jan 29 04:27 'year=2011'
drwxr-xr-x 14 root root 4096 Jan 29 04:27 'year=2010'


In [13]:
!ls -tl spark-warehouse/appl_stock/year\=2010/

total 48
drwxr-xr-x 2 root root 4096 Jan 29 04:27 'month=12'
drwxr-xr-x 2 root root 4096 Jan 29 04:27 'month=11'
drwxr-xr-x 2 root root 4096 Jan 29 04:27 'month=10'
drwxr-xr-x 2 root root 4096 Jan 29 04:27 'month=9'
drwxr-xr-x 2 root root 4096 Jan 29 04:27 'month=8'
drwxr-xr-x 2 root root 4096 Jan 29 04:27 'month=7'
drwxr-xr-x 2 root root 4096 Jan 29 04:27 'month=6'
drwxr-xr-x 2 root root 4096 Jan 29 04:27 'month=5'
drwxr-xr-x 2 root root 4096 Jan 29 04:27 'month=4'
drwxr-xr-x 2 root root 4096 Jan 29 04:27 'month=3'
drwxr-xr-x 2 root root 4096 Jan 29 04:27 'month=2'
drwxr-xr-x 2 root root 4096 Jan 29 04:27 'month=1'


In [14]:
!ls -tl spark-warehouse/appl_stock/year\=2010/month\=12/

total 4
-rw-r--r-- 1 root root 3027 Jan 29 04:27 part-00000-ee4ef65d-6e86-4c64-8880-aa44a099aad9.c000.snappy.parquet


### How to Read from Partitioned table

In [15]:
df = spark.read.table("appl_stock").where("year = 2016 and month = 12")

In [16]:
df.show(10)

+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|               Date|              Open|              High|       Low|     Close|  Volume|         Adj Close|year|month|
+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|2016-12-01 00:00:00|        110.370003|        110.940002|109.029999|109.489998|37086900|        109.017344|2016|   12|
|2016-12-02 00:00:00|109.16999799999999|        110.089996|108.849998|109.900002|26528000|        109.425578|2016|   12|
|2016-12-05 00:00:00|             110.0|        110.029999|    108.25|109.110001|34324500|108.63898700000001|2016|   12|
|2016-12-06 00:00:00|             109.5|        110.360001|109.190002|109.949997|26195500|109.47535800000001|2016|   12|
|2016-12-07 00:00:00|        109.260002|        111.190002|109.160004|111.029999|29998700|        110.550697|2016|   12|
|2016-12-08 00:00:00|        110

In [17]:
spark.sql("SELECT * FROM appl_stock WHERE year = 2016 and month = 12").show(10)

+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|               Date|              Open|              High|       Low|     Close|  Volume|         Adj Close|year|month|
+-------------------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|2016-12-01 00:00:00|        110.370003|        110.940002|109.029999|109.489998|37086900|        109.017344|2016|   12|
|2016-12-02 00:00:00|109.16999799999999|        110.089996|108.849998|109.900002|26528000|        109.425578|2016|   12|
|2016-12-05 00:00:00|             110.0|        110.029999|    108.25|109.110001|34324500|108.63898700000001|2016|   12|
|2016-12-06 00:00:00|             109.5|        110.360001|109.190002|109.949997|26195500|109.47535800000001|2016|   12|
|2016-12-07 00:00:00|        109.260002|        111.190002|109.160004|111.029999|29998700|        110.550697|2016|   12|
|2016-12-08 00:00:00|        110